Source: http://www.sthda.com/english/articles/37-model-selection-essentials-in-r/154-stepwise-regression-essentials-in-r/

**To run the R codes on Jupyter notebook, type “conda install -c r r-essentials” on your terminal – it will install the R kernel and some important R packages (e.g. dplyr, ggplot2, etc.)**


In [3]:
#Load the required packages 
library(tidyverse)
library(caret) 
library(leaps)

Warning message:
"package 'tidyverse' was built under R version 3.6.1"-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.2.1     v purrr   0.3.2
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   0.8.3     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
Warning message:
"package 'ggplot2' was built under R version 3.6.1"Warning message:
"package 'tidyr' was built under R version 3.6.1"Warning message:
"package 'readr' was built under R version 3.6.1"Warning message:
"package 'dplyr' was built under R version 3.6.1"Warning message:
"package 'stringr' was built under R version 3.6.1"Warning message:
"package 'forcats' was built under R version 3.6.1"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'caret' was built under R version 3.6.1"Loading required package: lattice

Attaching package: 'caret'

The following ob

In [4]:
#Load the dataset

df_housing <- read.csv("one_hot_df.csv", stringsAsFactors = FALSE)

In [5]:
#Get an overview of the train data set
#Click on white space below to expand out the code output

glimpse(df_housing)

Observations: 1,460
Variables: 274
$ X                        <int> 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...
$ SalePrice                <dbl> 12.24769, 12.10901, 12.31717, 11.84940, 12...
$ LotArea                  <dbl> -0.20714171, -0.09188637, 0.07347998, -0.0...
$ OverallQual              <dbl> 0.65147924, -0.07183611, 0.65147924, 0.651...
$ OverallCond              <dbl> -0.5171998, 2.1796278, -0.5171998, -0.5171...
$ YearBuilt                <dbl> 1.05099379, 0.15673371, 0.98475230, -1.863...
$ YearRemodAdd             <dbl> 0.8786681, -0.4295770, 0.8302146, -0.72029...
$ MasVnrArea               <dbl> 0.5141039, -0.5707501, 0.3259149, -0.57075...
$ BsmtFinSF1               <dbl> 0.57542484, 1.17199212, 0.09290718, -0.499...
$ BsmtUnfSF                <dbl> -0.94459061, -0.64122799, -0.30164298, -0....
$ TotalBsmtSF              <dbl> -0.459302541, 0.466464916, -0.313368755, -...
$ X1stFlrSF                <dbl> -0.79343379, 0.25714043, -0.62782603, -0.5...
$ X2ndFlrSF      

# Method 1: Using stepAIC()
Selects the best model by AIC (Akaike Information Criterion). 

In [ ]:
library(MASS)

#Fit your initial model to begin the regression 

full_model <- lm(SalePrice ~ ., data = df_housing)

There are three ways to do stepwise regression: (1) Backward, (2) Forward, (3) Stepwise. To select a particular model, you have to change the value in **direction**: (i) "both" (stepwise regression), (ii) "backward" (for backward regression) and "forward" (for forward selection). 

**trace** can be either TRUE or FALSE. TRUE means you want to see the results at each iteration, while FALSE does not show you each step, you just get the best final model at the end. If you have more variables then TRUE will give you a pretty long list of steps taken. 

In [ ]:
# stepwise_model <- stepAIC(SalePrice ~ ., direction = "both", trace = FALSE)

# backwards_model <- stepAIC(SalePrice ~ ., direction = "backward", trace = FALSE)

# forwards_model <- stepAIC(SalePrice ~ 1, direction = "forward", scope=formula(full_model), trace = FALSE)

In [ ]:
# Get the results of the final model 

summary(stepwise_model)

# Method 2: Using regsubsets()
Has tuning parameter **nvmax** specifying the maximal number of predictors to incorporate in the model. It returns multiple models with different size up to nvmax. You need to compare the performance of the different models for choosing the best one. 

regsubsets() has the option method, which can take the values “backward”, “forward” and “seqrep” (seqrep = sequential replacement, combination of forward and backward selections).

In [ ]:
models <- regsubsets(SalePrice ~ ., data = df_housing, nvmax = ?, method = 'seqrep')

summary(models)

# Method 3: CV and GridSearch

The train() function [caret package] provides an easy workflow to perform stepwise selections using the leaps and the MASS packages. It has an option named method, which can take the following values:

 - "leapBackward", to fit linear regression with backward selection
 - "leapForward", to fit linear regression with forward selection
 - "leapSeq", to fit linear regression with stepwise selection
 
You also need to specify the tuning parameter nvmax, which corresponds to the maximum number of predictors to be incorporated in the model.

For example, you can vary nvmax from 1 to 5. In this case, the function starts by searching different best models of different size, up to the best 5-variables model. That is, it searches the best 1-variable model, the best 2-variables model, …, the best 5-variables models.


We can use 5/10-fold cross-validation to estimate the average prediction error (RMSE) of each of the models. The RMSE statistical metric is used to compare the models and to automatically choose the best one, where best is defined as the model that minimize the RMSE.

In [ ]:
#Set seed for reproducibility
set.seed(42)

#Set up repeated k-fold cross-validation, indicate the number of folds you want
train_control <- trainControl(method = "cv", number = 10)

#Train the model, indicate the method of regression and range of nvmax numbers to try
step_model <- train(SalePrice ~ ., data = df_housing, 
                   method = "leapSeq",
                   tuneGrid = data.frame(nvmax = 10:270),
                   trControl = train_control)
step_model$results

Warning message in leaps.setup(x, y, wt = weights, nbest = nbest, nvmax = nvmax, :
"48  linear dependencies found"

**nvmax**: the number of variable in the model. For example nvmax = 2, specify the best 2-variables model

**RMSE** and **MAE** are two different metrics measuring the prediction error of each model. The lower the RMSE and MAE, the better the model.

**Rsquared** indicates the correlation between the observed outcome values and the values predicted by the model. The higher the R squared, the better the model.

In [1]:
#Display the best tuning values (nvmax) selected by the train() function

step_model$bestTune

ERROR: Error in eval(expr, envir, enclos): object 'step_model' not found


In [ ]:
#The function summary() reports the best set of variables for each model size 
summary(step_model$finalModel)